# Load Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pseudocode



this code below briefly explains how the whole process works

---------------
```python
data = raw_data()
#assign activity label
prevRow = null
foreach row in data:
  if row.ActivityLabel is null:
     row.ActivityLabel = prevRow.ActivityLabel
  prevRow = row
  
#group according to its activity
groups = grouping(data)

#apply sliding windows into each group
_ = []
foreach g in groups:
  sp = split(g, SECONDS_WINDOW)
  if sp is not empty:
      _ += sp
  
#convert each group to vector
vactors = []
for subg in _:
  for item in subg:
     vector[item.sensorID] += 1 if item.SensorValue == "ON"
     
  normalise(vector)
  vectors.append(vector)
  
return vectors
```
---------------


In [110]:
fileName = "annotated.sample"
dataDir = '../Data/twor.2009/'
columns = ["Date", "Time", "SensorID", "SensorValue", "ActivityLabel", "Status"]
raw = pd.read_csv(dataDir+fileName, sep=" ", names=columns, header=None)

In [111]:
# data = raw
data = raw[(raw["ActivityLabel"].notna()) | (raw["SensorValue"] != "OFF")]
data['DateTime'] =  data['Date'] + " "+data['Time']
data['Timestamp'] =  pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S.%f')


len(data)

534

In [112]:
def transform(d):
    g = []
    activity = None
    currGroup = []
    d["na"] = d[["ActivityLabel"]].isna()
    
    for i, row in d.iterrows():
        if not row["na"]:
            currGroup.append(row)
            activity = row["ActivityLabel"]
            if row['Status'] == "end":
                g.append(currGroup)
                currGroup = []
        else:
            row["ActivityLabel"] = activity
            row["Status"] = "working"
            currGroup.append(row)
            
    g.append(currGroup)
    return g

g = transform(data)

In [113]:
SECONDS_WINDOW = 15*60
def toVector(act, cols):
    if (len(act)==0):
        return []
    
    rows = []
    start = act[0]["Timestamp"].timestamp()
    activityLabel = act[0]["ActivityLabel"]
    actinwindow = {}
    for a in act:
        t = a["Timestamp"].timestamp()
        interval = ((t-start)//SECONDS_WINDOW)
        
        # consider only a second that when changes occur
        if interval in actinwindow:
            actinwindow[interval].append(a)
        else:
            actinwindow[interval] = [a]
    
    
    
    colIndex = {}
    for c in range(len(cols)):
        colIndex[cols[c]] = c
    
    vec = []
    for interval in actinwindow:
        v = np.zeros(len(cols))
        normalise = 0
        for a in actinwindow[interval]:
            if a["SensorValue"] in ["O", "ON", "OF", "ONF", "OPEN", "PRESENT"]:
                i = colIndex[a["SensorID"]]
                v[i] += 1

                normalise += 1
            elif a["SensorValue"] in ["OFFF", "OFF", "CLOSE", "ABSENT"]:
                continue
            else:
                # check isnumber https://stackoverflow.com/questions/354038/how-do-i-check-if-a-string-is-a-number-float
                value = a["SensorValue"].replace('F','',1)
                isNumberic = value.replace('.','',1).isdigit()
                if isNumberic:
                    value = float(value);
                    i = colIndex[a["SensorID"]]
                    v[i] += value
                    normalise += value
                else:
                    raise Exception('Unknown keyword: '+a["SensorValue"])
                    
            
        vec.append([value/normalise for value in v])
    
    rows = pd.DataFrame(vec, columns=cols)
    rows["ActivityName"] = activityLabel
    return rows

vColumns = np.insert(np.sort(raw.SensorID.unique()), 0, "ActivityName", axis=0);
vectors = pd.DataFrame(columns=vColumns)
for activity in g:
    v = toVector(activity, vColumns)
    if len(v) > 0:
        vectors = vectors.append(v, ignore_index=True)

vectors.head()

,ActivityName,M29,M30,M32,M33,M34,M35,M36,M37,M38,...,M42,M43,M44,M45,M46,M47,M48,M49,M50,M51
0,R2_work_at_computer,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.021277,0.829787,0.06383,0.0,0.0,0.06383,0.0,0.000000
1,R2_work_at_computer,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.972222,0.00000,0.0,0.0,0.00000,0.0,0.027778
2,R2_work_at_computer,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,1.000000,0.00000,0.0,0.0,0.00000,0.0,0.000000
3,R1_sleep,0.0,0.017857,0.035714,0.053571,0.428571,0.428571,0.035714,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.000000
4,R1_sleep,0.0,0.000000,0.000000,0.000000,0.400000,0.600000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.000000


In [114]:
# TODO: optimise by using aggregate function

# GroupID = 1
# def aggActivity(d):
#     global GroupID
#     na = d.isna()
#     if not na["ActivityLabel"] and d["Status"] == "end":
#         GroupID += 1
#         return GroupID - 1
#     return GroupID

# data['GroupID'] = data.apply(aggActivity, axis=1)

# SECONDS_WINDOW = 10*60

# StartInterval = None
# def splitTimestamp(d):
#     global StartInterval
#     global SECONDS_WINDOW
#     na = d.isna()
#     if not na["ActivityLabel"] and d["Status"] == "begin":
#         StartInterval = d["Timestamp"].timestamp()

#     t = d["Timestamp"].timestamp()
#     interval = ((t-StartInterval)//SECONDS_WINDOW)
#     return interval

# data['Interval'] = data.apply(splitTimestamp, axis=1)

# print(data.columns)
# g = data.groupby(["GroupID", "Interval", "SensorID"])

In [115]:
vectors.to_csv(dataDir+fileName+".feat", index=False)